## Initial Processing of Gravity Data

In [29]:
#Import required packages - 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import *
import pandas as pd
import pyproj
import rioxarray
import harmonica as hm
import xarray as xr
from scipy.interpolate import RegularGridInterpolator
import sys
sys.path.insert(0,'Required_packages')
from tidegravity import *
from scipy.interpolate import LinearNDInterpolator

### Read in the data
We define a function to read in the text file that is output by the gravity meter and then take the columns we want and put them in a pandas dataframe.

In [2]:
#This is a function to read in the text tile that is output but the gravity meter 
#- you don't need to understand exactly what it does but it basically reads the the file line by line and extracts certain bits for different data columns
linen=[]
station=[] 
alt=[]
grav=[]  
sd=[]
tiltx=[]
tilty=[]
temp=[]
etc=[]
dur=[]
rej=[]
t=[]
ts = []
corr_g=[]
keepdata=[]
epoch = []
keepitem=int
def readRawDataFile(filename):  
        """ read a raw ascii data text file extracted from CG5
     
        sometimes bad values are written by the CG5 soft, if such case happen,
        an error is raised and the user is asked for checking the data file 
        manually
        """    
   
        try:
            #essaye d'ouvrir le fichier
            fh = open(filename, 'r')
            i=0
            #PBAR = ProgressBar(total=len([1 for line in  open(filename, 'r')]),textmess='Load raw data')   
            #print "number of lines: %d"%len([1 for line in  open(filename, 'r')])
            #PBAR.show()
            for line in fh:    
                #PBAR.progressbar.setValue(i)
                i+=1
                # Clean line
                line = line.strip()
                # Skip blank and comment lines
                if (not line) or (line[0] == '/') or (line[0] == 'L'): continue
        	     #parse string line first with respect to '/' caracters (used in the date format), 
        	     #then with ':' (used for the time display), eventually with the classic ' '
                vals_temp1=line.split('/')
                vals_temp2=vals_temp1[0].split(':')
                vals_temp3=vals_temp2[0].split()                
                vals_temp4=vals_temp2[2].split()

                # fill object properties:
                linen.append(float(vals_temp3[0]))
                station.append(float(vals_temp3[1]))
                alt.append(float(vals_temp3[2]))
                grav.append(float(vals_temp3[3]))
                sd.append(float(vals_temp3[4]))
                tiltx.append(float(vals_temp3[5]))
                tilty.append(float(vals_temp3[6]))
                temp.append(float(vals_temp3[7]))
                etc.append(float(vals_temp3[8]))
                dur.append(int(vals_temp3[9]))
                rej.append(int(vals_temp3[10]))
                t.append(datetime(int(vals_temp4[3]),int(vals_temp1[1]),\
                int(vals_temp1[2]),int(vals_temp3[11]),int(vals_temp2[1]),\
                int(vals_temp4[0])))       
                t_tmp = '%04d-%02d-%02dT%02d:%02d:%02d'%\
                    (int(vals_temp4[3]),int(vals_temp1[1]),\
                    int(vals_temp1[2]),int(vals_temp3[11]),int(vals_temp2[1]),\
                    int(vals_temp4[0]))  
                ts.append(pd.Timestamp(t_tmp))
                epoch.append((pd.Timestamp(t_tmp) - pd.Timestamp("1970-01-01")).total_seconds())

                keepdata.append(1)                                                                                         
        except IOError:
            #si ça ne marche pas, affiche ce message et continue le prog
            print ('No file : %s' %(filename))            
        except ValueError:
            print ('pb at line %d : check raw data file'%(i))
        except IndexError:
            print ('pb at line %d : check raw data file: possibly last line?'%(i) )

In [3]:
#read in gravity data
readRawDataFile('Data/taku_grav_data.TXT') #since we have defined the function readRawDataFile in the above cell we can just call it here with our file name
#note that if you run this multiple times it will keep adding onto the bottom of the lists - so you'll end up multiple copys of the data in your dataframe

scintrex_data = pd.DataFrame({'grav':grav,'sta':station, 'timestamp':ts, 'sd':sd, 'tiltx':tiltx, 'tilty':tilty, 'temp':temp}) #here we define our pandas dataframe with the data columns we are interested in


### Read in and assign gps data

Our gps data is stored seperately so we also want to read that in and assign the correct positions and elevations to the stations by name.

In [5]:
#read in gps data
stations_gps = pd.read_csv('Data/grav_all_ppk_stations.csv', usecols=[0,3,4,5,6],skiprows=1,names=['Name', 'Elev_std', 'x_8N', 'y_8N', 'Elevation'])
#create columns with value -999 that we will overwrite with the correct value
scintrex_data['x_8N'] = -999
scintrex_data['y_8N'] = -999
scintrex_data['Elev'] = -999
scintrex_data['Elev_std'] = -999
for i in range(len(stations_gps)): #loop through all the stations in the gps data
    scintrex_data['x_8N'].iloc[scintrex_data.sta == stations_gps.Name.iloc[i]] = stations_gps['x_8N'].iloc[i]
    scintrex_data['y_8N'].iloc[scintrex_data.sta == stations_gps.Name.iloc[i]] = stations_gps['y_8N'].iloc[i]
    scintrex_data['Elev'].iloc[scintrex_data.sta == stations_gps.Name.iloc[i]] = stations_gps['Elevation'].iloc[i]
    scintrex_data['Elev_std'].iloc[scintrex_data.sta == stations_gps.Name.iloc[i]] = stations_gps['Elev_std'].iloc[i]

#at bases position is calculated from PPP processing of rinex file - we take the mean of the PPP position on multiple days
#camp 10 was station numbers between 100 and 120
scintrex_data['x_8N'].iloc[(scintrex_data['sta']>=100) & (scintrex_data['sta']<120)] = np.mean(stations_gps[(stations_gps['Name']>=100) & (stations_gps['Name']<120)].x_8N)
scintrex_data['y_8N'].iloc[(scintrex_data['sta']>=100) & (scintrex_data['sta']<120)] = np.mean(stations_gps[(stations_gps['Name']>=100) & (stations_gps['Name']<120)].y_8N)
scintrex_data['Elev'].iloc[(scintrex_data['sta']>=100) & (scintrex_data['sta']<120)] = np.mean(stations_gps[(stations_gps['Name']>=100) & (stations_gps['Name']<120)].Elevation)
scintrex_data['Elev_std'].iloc[(scintrex_data['sta']>=100) & (scintrex_data['sta']<120)] = np.mean(stations_gps[(stations_gps['Name']>=100) & (stations_gps['Name']<120)].Elev_std)

#transform to wgs84 lat long
#define transformations
wgs84 = 'epsg:4326' # Global lat-lon coordinate system - to be used for tide and latitude corrections
utm8N = 'epsg:26908' #utm8n - nad 83 - the projected coordinate system being used
utm8N2wgs = pyproj.Transformer.from_crs(utm8N, wgs84)
scintrex_data['Lat'], scintrex_data['Long'] = utm8N2wgs.transform(scintrex_data['x_8N'].values, scintrex_data['y_8N'].values)



/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_65358/2273284778.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scintrex_data['x_8N'].iloc[scintrex_data.sta == stations_gps.Name.iloc[i]] = stations_gps['x_8N'].iloc[i]
/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_65358/2273284778.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scintrex_data['y_8N'].iloc[scintrex_data.sta == stations_gps.Name.iloc[i]] = stations_gps['y_8N'].iloc[i]
/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_65358/2273284778.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

### Earth tide


In [6]:
tide_calc = solve_longman_tide(scintrex_data.Lat.values, scintrex_data.Long.values, scintrex_data.Elev.values, pd.DatetimeIndex(scintrex_data.timestamp.values))
scintrex_data['tide_calc'] = tide_calc[2]
scintrex_data['tide_corr_grav'] = scintrex_data['grav'] + scintrex_data['tide_calc'] #we add the calculated tide correction to get the tide corrected gravity


### Average Measurements
We have at least 4 measurements at each location, so we want to average over them so we just have one value at each location

In [7]:
st_names = scintrex_data['sta'].unique()
station_avgs = pd.DataFrame(st_names, columns=['sta'])

#create lists for values to extract from data
grav_avg = []
grav_std = []
center_dt = []
lat = []
long = []
x_8N = []
y_8N = []
elev = []
elev_std = []

for n in st_names:
    #grav_vals = []
    select = scintrex_data[scintrex_data.sta == n]
    grav_avg.append(np.mean(select.tide_corr_grav))
    grav_std.append(np.std(select.tide_corr_grav))
    center_dt.append(select.timestamp.iloc[0] + (select.timestamp.iloc[-1] - select.timestamp.iloc[0])/2)
    lat.append(np.mean(select.Lat))
    long.append(np.mean(select.Long))
    x_8N.append(np.mean(select.x_8N))
    y_8N.append(np.mean(select.y_8N))
    elev.append(np.mean(select.Elev))
    elev_std.append(np.mean(select.Elev_std))
#create new dataframe with all extracted values
data_avgs = pd.DataFrame(zip(st_names, grav_avg, grav_std, center_dt, lat, long, x_8N, y_8N, elev, elev_std), columns=['sta','grav_avg', 'grav_std', 'Datetime', 'Lat', 'Long', 'x_8N', 'y_8N','Elev', 'Elev_std'])

In [10]:
#Now we split the data from the base station to calculate the drift
data_avgs['secs_elapsed'] = (data_avgs['Datetime'] - data_avgs['Datetime'].iloc[0])/pd.Timedelta(seconds=1)

c10_base = data_avgs.loc[(data_avgs['sta']>=100) & (data_avgs['sta']<120)].reset_index(drop=True)


### Drift 
We need to calculate the effect of the drift of the gravity meter on the measurements - correct with linear relationship

In [11]:
a, b = np.polyfit(c10_base.secs_elapsed, c10_base.grav_avg, 1)
c10_base['linear_drift_corr'] = c10_base.grav_avg - (c10_base.secs_elapsed*a +b)
data_avgs['linear_drift_corr'] = data_avgs.grav_avg - (data_avgs.secs_elapsed*a +b)

In [12]:

stations = data_avgs.loc[(data_avgs['sta']<100) | (data_avgs['sta']>=200) | (data_avgs['sta']==102)].reset_index(drop=True) #stations all non base locations plus c10 at start of day 1 - part of profile 4
stations['linear_drift_corr'].iloc[0] = np.mean(c10_base.linear_drift_corr) #set c10 to be mean of all drift corrected measurements there
stations['grav_anom'] = stations['linear_drift_corr'] - np.mean(c10_base.linear_drift_corr) #make all measurements relative to mean at c10

/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_65358/130087068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['linear_drift_corr'].iloc[0] = np.mean(c10_base.linear_drift_corr) #set c10 to be mean of all drift corrected measurements there


### Latitude correction

In [13]:
stations['theo_grav_0'] =  9.7803267715*(1 + 0.0052790414*np.sin(np.deg2rad(stations.Lat))**2 
                                            + 0.0000232718*np.sin(2*np.deg2rad(stations.Lat))**4
                                            + 0.0000001262*np.sin(2*np.deg2rad(stations.Lat))**6
                                            + 0.0000000007*np.sin(2*np.deg2rad(stations.Lat))**8)*1e5


In [14]:
# Error on drift corrected data - root sum of squares combination of station and base standard deviations
#base std is the standard deviation of the C10 base station measurments after linear drift correction
base_std = np.std(c10_base.linear_drift_corr)
stations['drift_corr_grav_uc'] = np.sqrt(stations['grav_std']**2 + base_std**2)

### Free air anomaly

In [15]:
stations['fac'] = stations.Elev * 0.3086 #( R. E. Sheriff, Encyclopedic Dictionary of Exploration Geophysics, 2nd ed., (Society of Exploration Geophysicists, 1984), p. 141.)
stations['faa_error'] = np.sqrt((stations.Elev_std * -0.3086)**2 + (stations.drift_corr_grav_uc)**2)

### Bouguer anomaly
This is done as a simple slab correction with an additional component from the terrain correction.

#### Bouguer Slab

In [16]:
stations['boug_slab_corr_2700'] = 2*np.pi*2700*stations.Elev*6.67e-11*1e5
stations['ba_error'] = np.sqrt((0.196*stations.Elev_std)**2  + (stations.drift_corr_grav_uc)**2)

#### Terrain Correction

In [24]:
#import the tif file of the arctic dem surface elevations and regrid at 100 m spacing 
def open_raster(file): #create a function to read in the dem in tif format and convert it to an xarray which is used by harmonica
    #note function returns a dataArray and a Dataset which are different things - we use the dataArray - and also an array of x,y,z values
    geotiff_da = rioxarray.open_rasterio(file)
    geotiff_ds = geotiff_da.to_dataset('band') # Covert our xarray.DataArray into a xarray.Dataset
    geotiff_ds = geotiff_ds.rename({1: 'topo'}) # Rename the variable to a more useful name in the dataset

    geo_tiff_topo = geotiff_da[0] # in the dataArray select just the first variable which is the topography

    [x_topo, y_topo] = np.meshgrid(geo_tiff_topo.x.values, geo_tiff_topo.y.values) # the x and y values are read in as a single array for each and we want to create a grid of all values
    z_topo = geo_tiff_topo.values # defining the z component as the topography

    topo_xyz = np.c_[x_topo.ravel(), y_topo.ravel(), z_topo.ravel()] # we combine the x,y and z information into one array to be returned as a standard array

    topo_xr = xr.DataArray(geo_tiff_topo.values, # create the DataArray
    coords={'y': y_topo[:,0],'x': x_topo[0,:]}, 
    dims=["y", "x"])

    return topo_xr #returns the dataset, numpy array and dataArray


surf_xr = open_raster('Data/arctic_dem_plus_10m_nad83.tif')
surf_xr = surf_xr.reindex(y=surf_xr.y[::-1]) #invert y axis so it is negative to positive
mask = (surf_xr < 0).values
surf_xr = surf_xr.where(mask == False, 0)

interp = RegularGridInterpolator((surf_xr.y.values, surf_xr.x.values), surf_xr.values) # create an interpolator function
dem_xx = np.linspace(min(surf_xr.x.values), max(surf_xr.x.values), int(len(surf_xr.x.values)/10)) # create an array of x values that we want to resample onto - in this case I just halfed the number of values
dem_yy = np.linspace(min(surf_xr.y.values), max(surf_xr.y.values), int(len(surf_xr.y.values)/10))
xx, yy = np.meshgrid(dem_xx, dem_yy) # mesh grid the 1D arrays
bb = interp((yy, xx)) # use the interpolator function on these new x and y positions

#create a new DataArray
surf_xr_resamp = xr.DataArray(bb,
coords={'y': dem_yy,'x': dem_xx}, 
dims=["y", "x"])

In [26]:
x_coords_sta = xr.DataArray(stations['x_8N']) #turn the x and y into dataArrays so they can be used to sample the dem
y_coords_sta = xr.DataArray(stations['y_8N'])

In [27]:
#define function for calculating gravity from density and surface elevation distribution
def prism_calc(density, surface):
    density_xr = surface.copy() # create a density data array
    density_xr.values[:] = density  # replace every value for the density of the topography
    prisms = hm.prism_layer( # this is where we create the model of the topography, by creating a layer of prisms with varying elevation based on the dem
        (surface.x.values, surface.y.values),
        surface=surface,
        reference=0,
        properties={"density": density_xr})
    
    dem_elev_sta = surface.sel(x=x_coords_sta, y=y_coords_sta, method='nearest') #find the dem elevation at the station and base locations

    prisms_g_sta = prisms.prism_layer.gravity((stations['x_8N'], stations['y_8N'], dem_elev_sta.values), field="g_z") #calculate the 3D gravity from the dem
    terr_corr_sta = dem_elev_sta.values*2*np.pi*density*6.67e-11*1e5 - prisms_g_sta #subtract from the bouguer slab correction with the dem values

    return terr_corr_sta

In [30]:
#calculate uncertainty on terrain correction - take ~15 minutes, if want values from prior run uncomment first line in next cell
n_runs = 100
terrain_corrs_sta = np.empty((n_runs, len(stations)))

for i in range(n_runs):
    perturb = s = np.random.normal(0, 1, len(surf_xr_resamp.y)*len(surf_xr_resamp.x)) #perturb dem with standard deviation 1m
    perturb_reshape = perturb.reshape(len(surf_xr_resamp.y), len(surf_xr_resamp.x)) 
    surf_pert = surf_xr_resamp.copy() + perturb_reshape #add pertubation
    den = np.random.randint(2650, 2751) #select density from distribution of reasonable values
    terrain_corrs_sta[i,:] = prism_calc(den, surf_pert) #calculate terrain correction with density and perturbed dem
terr_corr_error = np.std(terrain_corrs_sta, axis=0)

In [45]:
# terr_corr_error = np.array([0.06907342, 0.02967226, 0.02910847, 0.02806514, 0.02995369,
#        0.02823702, 0.02887071, 0.03164564, 0.03570411, 0.03164564,
#        0.02967875, 0.03207385, 0.02876628, 0.02830362, 0.02848493,
#        0.02963774, 0.03362601, 0.03179451, 0.02889714, 0.02994969,
#        0.03133625, 0.03121335, 0.08700415, 0.03176602, 0.0382349 ,
#        0.03196948, 0.03279301, 0.03528296, 0.03990372, 0.03336947,
#        0.05482707, 0.03443134, 0.03338897, 0.03501866, 0.03617349,
#        0.03608689, 0.0373075 , 0.03293801, 0.06750231, 0.04261558,
#        0.03498449, 0.07284373, 0.07414116, 0.0732809 , 0.03932213,
#        0.04617974, 0.03267181, 0.0379076 , 0.0337315 , 0.04876516])
stations['ba_terr_error'] = np.sqrt(stations['ba_error']**2 + terr_corr_error**2)

In [39]:
stations['terr_corr_2700'], c10_base['terr_corr_2700']  = prism_calc(2700, surf_xr_resamp) #calculate terrain correction with density and perturbed dem

In [40]:
stations['fa_grav_theo'] = -stations['theo_grav_0'] + stations['fac']
stations['fa_grav_theo_rel'] = stations['fa_grav_theo'] - stations['fa_grav_theo'].iloc[0]
stations['fa_anom'] = stations['grav_anom'] + stations['fa_grav_theo_rel']

stations['grav_theo_2700'] = -stations['theo_grav_0'] + stations['fac'] - stations['boug_slab_corr_2700'] + stations['terr_corr_2700']
stations['grav_theo_rel'] = stations['grav_theo_2700'] - stations['grav_theo_2700'].iloc[0]
stations['boug_anom_2700'] = stations['grav_anom'] + stations['grav_theo_rel']

### Repeat measurements - error estimation and averaging

In [41]:
# repeat measurments differences
rep_meas = [1, 11, 7, 71, 4, 21, 210, 212, 305, 409, 403, 408] #numbers of repeat measurement stations

diff_max = stations['boug_anom_2700'][stations['sta']==rep_meas[0]].values - stations['boug_anom_2700'][stations['sta']==rep_meas[1]].values #maximum repeat measurement
stations['rep_error_max'] = diff_max[0]

data_rep_avg = stations.copy() #create new dataframe to average repeat measurements
for i in range(0,len(rep_meas),2):
    # data_rep_avg['faa'][data_rep_avg['sta']==rep_meas[i]] =  (data_rep_avg['faa'][data_rep_avg['sta']==rep_meas[i]].values + data_rep_avg['faa'][data_rep_avg['sta']==rep_meas[i+1]].values)/2
    data_rep_avg['boug_anom_2700'][data_rep_avg['sta']==rep_meas[i]] =  (data_rep_avg['boug_anom_2700'][data_rep_avg['sta']==rep_meas[i]].values + data_rep_avg['boug_anom_2700'][data_rep_avg['sta']==rep_meas[i+1]].values)/2
    data_rep_avg = data_rep_avg.drop(index = data_rep_avg[data_rep_avg['sta']==rep_meas[i+1]].index)


/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_65358/1622556540.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rep_avg['boug_anom_2700'][data_rep_avg['sta']==rep_meas[i]] =  (data_rep_avg['boug_anom_2700'][data_rep_avg['sta']==rep_meas[i]].values + data_rep_avg['boug_anom_2700'][data_rep_avg['sta']==rep_meas[i+1]].values)/2


In [42]:
#rss the measurement error to the repeat meas error
data_rep_avg['error'] = np.sqrt(data_rep_avg['rep_error_max']**2 + data_rep_avg['ba_terr_error']**2)

### Seperate out Profiles
Create seperate dataframes for each of the Profiles and calculate distance along them

In [47]:

#seperate out the profile 4 data - day 1 station numbers plus c10 base station
profile4 = data_rep_avg[(data_rep_avg.sta < 20) | (data_rep_avg.sta == 102)].copy()
profile4 = profile4.sort_values(by='x_8N', ascending=False).reset_index(drop=True)
profile4['dist'] = 0
for i in range(1, len(profile4)):
    profile4['dist'].iloc[i] = profile4['dist'].iloc[i-1] + np.sqrt((profile4['x_8N'].iloc[i] - profile4['x_8N'].iloc[i-1])**2 +(profile4['y_8N'].iloc[i] - profile4['y_8N'].iloc[i-1])**2 )

#seperate longitudinal profile - day 2 and 3 measurements plus station 4 which is at intersection with profile 4
longa = data_rep_avg[(data_rep_avg.sta > 19) & (data_rep_avg.sta < 99) | (data_rep_avg.sta == 4) | (data_rep_avg.sta > 199) & (data_rep_avg.sta < 399)].copy()
longa = longa.sort_values(by='y_8N', ascending=True).reset_index(drop=True)
longa['dist'] = 0
for i in range(1, len(longa)):
    longa['dist'].iloc[i] = longa['dist'].iloc[i-1] + np.sqrt((longa['x_8N'].iloc[i] - longa['x_8N'].iloc[i-1])**2 +(longa['y_8N'].iloc[i] - longa['y_8N'].iloc[i-1])**2 )

#seperate profile 7a - day 4 measurments plus 305 which is where long A intersects
profile7a = data_rep_avg[(data_rep_avg.sta> 399) | (data_rep_avg.sta == 305)].copy()
profile7a = profile7a.sort_values(by='x_8N', ascending=False).reset_index(drop=True)
profile7a['dist'] = 0
for i in range(1, len(profile7a)):
    profile7a['dist'].iloc[i] = profile7a['dist'].iloc[i-1] + np.sqrt((profile7a['x_8N'].iloc[i] - profile7a['x_8N'].iloc[i-1])**2 +(profile7a['y_8N'].iloc[i] - profile7a['y_8N'].iloc[i-1])**2 )


/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_65358/3450042324.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profile4['dist'].iloc[i] = profile4['dist'].iloc[i-1] + np.sqrt((profile4['x_8N'].iloc[i] - profile4['x_8N'].iloc[i-1])**2 +(profile4['y_8N'].iloc[i] - profile4['y_8N'].iloc[i-1])**2 )
/var/folders/6g/r2y8czh94x50dtz0hyff18nr0000gq/T/ipykernel_65358/3450042324.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longa['dist'].iloc[i] = longa['dist'].iloc[i-1] + np.sqrt((longa['x_8N'].iloc[i] - longa['x_8N'].iloc[i-1])**2 +(longa['y_8N'].iloc[i] - longa['y_8N'].iloc[i-1])**2 )
/var/folders/6g/r

In [48]:
#export profiles
profile4.to_csv('Data/prof4_meas.csv', index=False)
profile7a.to_csv('Data/prof7a_meas.csv', index=False)
longa.to_csv('Data/longa_meas.csv', index=False)